In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scipy as sp
import numpy as np
import torch
from torch import nn, functional as F
from torch.utils.data import Dataset, DataLoader
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
MIN_IN_DAY = 1440
CUTOFF = 172800

In [ ]:
from zipfile import ZipFile
myzip=ZipFile('/content/drive/MyDrive/Colab Notebooks/load/data.zip')
f=myzip.open('data.csv')
df=pd.read_csv(f)
f.close()
myzip.close()

In [ ]:
df = pd.read_csv('processed_data.csv')
df

In [ ]:
daily = np.array(df.iloc[:, 1:])[:CUTOFF].T  # 346 residents, 172800 min
daily = daily.reshape((daily.shape[0], -1, MIN_IN_DAY))  # 346 residents, 120 days, 1440 minutes
daily = np.nanmean(daily, axis=1)  # 346 residents, 1440 minutes

In [ ]:
@widgets.interact(resident=(0, daily.shape[0]-1))
def plot_daily_load(resident=31):
    plt.plot(daily[resident])
    plt.show()

In [ ]:
def normalized(load):
    peak = load.max(axis=1)[:, None]
    trough = load.min(axis=1)[:, None]
    diff = peak - trough
    diff[diff == 0.] = 1.
    normalized = (load - trough) / diff
    return normalized

In [ ]:
normalized_daily = normalized(daily)

In [ ]:
@widgets.interact(resident=(0, normalized_daily.shape[0]-1))
def plot_normalized_daily(resident=217):
    plt.plot(normalized_daily[resident])
    plt.show()

In [ ]:
#
#
# Training
#
#

In [ ]:
# Dataset
class DS(Dataset):
    def __init__(self, data, sep, train=True, device='cuda'):
        super().__init__()
        if train:
            self.data = torch.Tensor(data[:sep]).to(device)
        else:
            self.data = torch.Tensor(data[sep:]).to(device)
        self.data.unsqueeze_(1)
    
    def __getitem__(self, i):
        return self.data[i]
    
    def __len__(self):
        return self.data.shape[0]

In [ ]:
# Autoencoder with MLP

In [ ]:
class AE_MLP(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        encoder = []
        decoder = []
        act = cfg['activation']
        
        # encoder
        for i in range(len(cfg['encoder']) - 1):
            cin, cout = cfg['encoder'][i], cfg['encoder'][i+1]
            encoder.append(nn.Linear(cin, cout))
            encoder.append(act)

        # decoder
        for i in range(len(cfg['decoder']) - 1):
            cin, cout = cfg['decoder'][i], cfg['decoder'][i+1]
            decoder.append(nn.Linear(cin, cout))
            decoder.append(act)

        self.encoder = nn.Sequential(*encoder)
        self.decoder = nn.Sequential(*decoder)
        
    def forward(self, x):
        latent = self.encoder(x)
        recon = self.decoder(latent)
        return latent, recon

In [ ]:
cfg = {
    'activation': nn.ReLU(),
    'encoder': [1440, 256, 64, 4],
    'decoder': [4, 64, 256, 1440]
}
model = AE_MLP(cfg)
model.to(device)

In [ ]:
torch.save(model.state_dict(), 'models/AE_CNN_sigmoid_d=2.pt')

In [ ]:
# Autoencoder with CNN

In [ ]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.shape[0], *self.shape)

class AE_CNN(nn.Module):
    def __init__(self, cfg, load_dict=None, device='cuda'):
        super().__init__()
        self.device = device
        act = cfg['activation']
        d = cfg['latent_dim']
        
        # encoder
        enc = [
            # 1440 -> 288
            nn.Conv1d(1, 16, 5, padding=2),
            nn.MaxPool1d(kernel_size=5),
            act,
            
            # 288 -> 72
            nn.Conv1d(16, 32, 4, padding=2),
            nn.MaxPool1d(kernel_size=4),
            act,
            
            # 32*72
            nn.Flatten(),
            # 32*72 -> d (fully connected)
            nn.Linear(32*72, d),
#             nn.ReLU(),
        ]

        # decoder
        dec = [
            # d -> 32*72 (fully connected)
            nn.Linear(d, 32*72),
#             nn.ReLU(),
            # 72
            View((32, 72)),

            # 32 -> 96
            nn.Conv1d(32, 16, 4, padding=2),
            nn.Upsample(288, mode='linear'),
            act,
            
            # 96 -> 288
            nn.Conv1d(16, 1, 5, padding=2),
            nn.Upsample(1440, mode='linear'),
#             nn.Tanh()
            nn.Sigmoid(),
        ]
        
        self.encoder = nn.Sequential(*enc)
        self.decoder = nn.Sequential(*dec)

        self.to(device)
        
        if load_dict is not None:
            self.load_state_dict(
                torch.load(load_dict, map_location=self.device)
            )
        
    def forward(self, x):
        latent = self.encoder(x)
        recon = self.decoder(latent)
        return latent, recon
        return latent

In [ ]:
cfg = {
    'latent_dim': 2,
    'activation': nn.ReLU(),
}

# model_dict = 'models/AE_CNN_sigmoid_d=8.pt'
model_dict = None
model = AE_CNN(cfg, model_dict, device)

In [ ]:
# model(test_data).shape
l, r = model(test_data)
l.shape, r.shape

In [ ]:
# Training setup
# At each epoch, randomly shuffle the daily loads, then feed in the network sequentially.
ntraindata = 250
epoch = 1000
lr = 1e-3
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss()
bsz = 125
train_dataset = DS(normalized_daily, ntraindata, train=True, device=device)
test_dataset = DS(normalized_daily, ntraindata, train=False, device=device)
train_loader = DataLoader(train_dataset, batch_size=bsz, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
test_data = iter(test_loader).next()

train_loss = []
test_loss = []

In [ ]:
def vis_epoch(train_loss, val_loss, val_data, recon_data, rows=4, cols=4, epoch=-1, saveat=None):
    plt.rcParams.update(plt.rcParamsDefault)
    fig = plt.figure(figsize=(20, 8))
    gs = gridspec.GridSpec(1, 2)
    ax0 = fig.add_subplot(gs[0])
    ax0.set_xlabel('Epochs', fontsize=11, weight='semibold')
    ax0.set_ylabel('MSE', fontsize=11, weight='semibold')
    ax0.plot(train_loss, color='#fc5a50', label='Train', alpha=0.9)
    ax0.plot(val_loss, color='#029386', label='Validation', alpha=0.9)
    ax0.spines['right'].set_visible(False)
    ax0.spines['top'].set_visible(False)
    ax0.set_ylim([0., 0.1])
    ax0.legend(prop={'size': 11,})
    grid = gridspec.GridSpecFromSubplotSpec(rows, cols, subplot_spec=gs[1])

    for i in range(rows):
        for j in range(cols):
            ax = fig.add_subplot(grid[i, j])
            idx = i * rows + j
            ax.plot(test_data[idx][0].detach().cpu(), color='#2b7ce0', alpha=0.6)
            ax.plot(recon_data[idx][0].detach().cpu(), color='#e02b5b', alpha=0.9) # # #e03d2b
            ax.set_ylim([0., 1.])
            ax.axis('off')
    if saveat is not None:
        plt.savefig(f'{saveat}/{epoch}.png', format='png')
    plt.show()


# Training loops
for e in range(epoch):
    for ibatch, batch in enumerate(train_loader):
        optim.zero_grad()
        latent, recon = model(batch)
        loss = loss_fn(recon, batch)
        loss.backward()
        optim.step()

    if e % 1 == 0:
        train_loss.append(loss.item())
        # evaluate network
        with torch.no_grad():
            latent, recon = model(test_data)
            loss = loss_fn(recon, test_data)
            test_loss.append(loss.item())
            vis_epoch(train_loss, test_loss, test_data, recon, 
                      rows=4, cols=4, epoch=e, saveat='/home/jamie/Desktop/graphics')        

In [ ]:
#
#
# WILLLLD EXPERIMENTS!
#
#

In [ ]:
data = DS(normalized_daily, sep=0, train=False, device=device).data
data_c = data.detach().cpu().numpy()[:, 0]

latent, pred = model(data)
latent = latent.detach().cpu().numpy()
pred = pred.detach().cpu().numpy()[:, 0]
latent.shape, pred.shape

In [ ]:
@widgets.interact(resident=(0, data.shape[0]-1))
def plot_reconstruction(resident=data.shape[0]//2):
    plt.plot(data_c[resident])
    plt.plot(pred[resident])
    plt.show()
    print(np.around(latent[resident], 5))

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
latent.shape

In [ ]:
clusters = 7

In [ ]:
kmeans = KMeans(n_clusters=clusters, random_state=2000).fit(latent)

In [ ]:
kmeans.labels_
kmeans.cluster_centers_

In [ ]:
def decode(latent_vec: np.array, model) -> np.array:
    latent_tensor = torch.Tensor(latent_vec[None,:,None]).to(device)
    with torch.no_grad():
        decoded = model.decoder(latent_tensor.permute(0, 2, 1))
    return decoded.cpu().numpy()[0, 0, :]


In [ ]:
@widgets.interact(Cluster=(0, clusters-1))
def plot_cluster(Cluster):
    import math
    resd = (kmeans.labels_==Cluster)
    load = data_c[resd]
    mean = load.mean(axis=0)
    centroid = kmeans.cluster_centers_[Cluster]
    decoded_mean = decode(centroid, model)
    print(f'{load.shape[0]} residents')
    alpha = 1.0 / math.sqrt(load.shape[0]) / 2.
    plt.plot(load.T, alpha=alpha)
    plt.plot(mean, c='red')
    plt.plot(decoded_mean, c='green')
    plt.show()
    print('Latent centroid:')
    print(np.around(centroid, 5))

In [ ]:
def display_cluster(model, latent_dim=8, figsize=(15, 6)):
    '''
    model: the neural network used for decoding
    latent_dim int: the dimention of latent vectors
    '''
    def extract_value(**latents):
        latent_data = np.array(list(latents.values()))
        reconstruction = decode(latent_data, model)
        plt.figure(figsize = figsize)
        plt.plot(reconstruction)
        plt.ylim((0,1))

    slider_list = [widgets.FloatSlider(value = 0, min = -5, max = 5, 
                                       step = 0.01, drscription = 'latent variable' + str(i),
                                       orientation = 'horizontal') for i in range(latent_dim)]
    ui = widgets.GridBox(slider_list,layout = widgets.Layout(grid_template_columns="repeat(3, 300px)"))
    arg_dict = {str(idx):slider for idx, slider in enumerate(slider_list)}
#     arg_dict['model'] = model
    out = widgets.interactive_output(extract_value, arg_dict)
    display(ui,out)
    

In [ ]:
latent_dim = 4
cfg = {
    'latent_dim': latent_dim,
    'activation': nn.ReLU(),
}

model_dict = '/content/drive/MyDrive/Colab Notebooks/load/models/AE_CNN_sigmoid_d={}.pt'.format(latent_dim)
#model_dict = None
model = AE_CNN(cfg, model_dict, device)
display_cluster(model, latent_dim=latent_dim)

In [ ]:
#### PCA dimension reduction

latent_dim = 2

cfg = {
    'latent_dim': latent_dim,
    'activation': nn.ReLU(),
}

model_dict = 'models/AE_CNN_sigmoid_d={}.pt'.format(latent_dim)
#model_dict = None
model = AE_CNN(cfg, model_dict,device = device)
model.to(device)


data = DS(normalized_daily, sep=0, train=False).data
latent, _ = model(data)
latent = latent.detach().cpu().numpy()



In [ ]:
from sklearn.cluster import KMeans
nclusters = 6
kmm_latent = KMeans(n_clusters = nclusters, random_state = 2000).fit(latent)
kmm_original = KMeans(n_clusters = nclusters, random_state = 2000).fit(normalized_daily)

In [ ]:
# pca use latent variable
from sklearn.decomposition import PCA
pca = PCA(n_components = 2).fit(latent)
pc = pca.transform(latent)
plt.scatter(pc[:,0],pc[:,1],c = kmm_latent.labels_,cmap = 'coolwarm')

In [ ]:
# pca use original data
pca = PCA(n_components = 2).fit(normalized_daily)
pc = pca.transform(normalized_daily)
plt.scatter(pc[:,0],pc[:,1],c = kmm_original.labels_,cmap = 'coolwarm')

In [ ]:
from sklearn.manifold import TSNE
# tSNE
embeded = TSNE(n_components = 2,perplexity = 80, learning_rate = 20,random_state=2000).fit_transform(latent)
plt.scatter(embeded[:,0], embeded[:,1], c = kmm_latent.labels_, cmap = 'coolwarm')

In [ ]:
# tSNE
embeded = TSNE(n_components = 2,perplexity = 80, learning_rate = 20,random_state=2000).fit_transform(normalized_daily)
plt.scatter(embeded[:,0], embeded[:,1], c = kmm_original.labels_, cmap = 'coolwarm')

In [ ]:
# elbow plot of kmeans using original and latent data
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from collections import defaultdict
# plot elbow plot
def elbow_plot(X,n_clusters = np.arange(1,40,2)):
    distortions = []
    for n in n_clusters:
        kmmodel = KMeans(n)
        kmmodel.fit(X)
        dis = 0
        for i in range(n):
            cluster_data = X[kmmodel.labels_ == i]
            centroid = cluster_data.mean(0)
            dis += ((cluster_data - centroid)**2).sum()
        distortions.append(dis/X.shape[0])
        # distortions.append(sum(np.min(cdist(X, kmmodel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
    plt.plot(n_clusters, distortions, 'bx-')
    plt.xticks(n_clusters)
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method')
    return distortions

def elbow_plot_latent(X,model,n_clusters = np.arange(1,40,2)):
    distortions = []
    data = DS(X, sep=0, train=False, device=device).data
    latent,_ = model(data)
    latent = latent.detach().cpu().numpy()
    for n in n_clusters:
        kmmodel = KMeans(n)
        kmmodel.fit(latent)
        dis = 0
        for i in range(n):
            cluster_data = X[kmmodel.labels_ == i]
            centroid = cluster_data.mean(0)
            dis += ((cluster_data - centroid)**2).sum(1).sum()
        distortions.append(dis/X.shape[0])
    plt.plot(n_clusters, distortions, 'rx-')
    plt.xticks(n_clusters)
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method')
    return distortions

In [ ]:
latent_elbow = elbow_plot_latent(normalized_daily,model,np.arange(1,20))
normal_elbow = elbow_plot(normalized_daily,np.arange(1,20))

In [ ]:
elbow_plot(normalized_daily)